In [1]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [5]:
wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")
btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)
print(wallet.get_address())
print(btest_wallet.get_address())

71217.214579
71217.214579
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db


In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server
# for the first time since reinitializing.

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [7]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [8]:
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))

In [9]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
print(get_scores_json(contracts))

core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 0
Failure: {'code': 10, 'message': 'Out of step: contractSet'}

core_contracts/dividends.zip
Status: 0
Failure: {'code': 6, 'message': 'Unknown param: key=_governance value=cxa2a26534c72a6ef4b8169036fed1b287e280969f'}

core_contracts/reserve.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 0
Failure: {'code': 6, 'message': 'Unknown param: key=_governance value=cxa2a26534c72a6ef4b8169036fed1b287e280969f'}

core_contracts/daofund.zip


ValueError: Invalid path core_contracts/daofund.zip

In [56]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['oracle']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

FileNotFoundError: [Errno 2] No such file or directory: 'token_contracts'

In [55]:
# Cell 10
# 1. Deposit collateral to fresh wallet
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.
# 3a. Mints 200 ICD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

send_tx('loans', 782769 * ICX // 1000, 'addCollateral', {'_asset': '', '_amount': 0}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'add_bad_test_position', {}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})

------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85', 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5', '0x2a6f1a22364bbe8000'], 'data': []} 

{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['IScoreClaimedV2(Address,int,int)', 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'], 'data': ['0x11f4b678367a3025600', '0x498c22b0ffe831c0']} 

{'scoreAddress': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x2a6f1a22364bbe8000'], 'data': ['0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f7365

In [57]:
# Cell 11
# 4a. If the account does not yet have a standing of 'Liquidate' add another loan of '0xad78ebc5ac6200000'.

send_tx('loans', 0, 'add_bad_test_position', {}, btest_wallet)
print(call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()}))

------------------------------------------------------------------------------------------------------------------
Calling add_bad_test_position, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4c1beaa71b9377100c810c46059ddf5f3da37602', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0xad78ebc5ac6200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']} 

{'scoreAddress': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain

In [19]:
# 5. Check the account standing after depositing collateral - Should be 'No Debt'.
# 6. Liquidate the account position.
# 7. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.
# 8. Checking the debts should show the sum of the borrower debt and the bad debt equals the total supply.

call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'liquidate', {'_owner': btest_wallet.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
call_tx('loans', 'checkDebts', {})

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x3', 'created': '0x5bd90ea53fda8', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x64', 'snaps_length': '0x1', 'last_snap': '0x64', 'first day': '0x64', 'assets': {'sICX': '0x2a6f1a22364bbe8000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'No Debt'}
------------------------------------------------------------------------------------------------------------------
Calling liquidate, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
---------------------------------------------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxf288f27511d719727ef9a17cec39971353b91519', 'indexed': ['Mint(Address,int,bytes)', 'cxf288f27511d719727ef9a17cec39971353b91519'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxf288f27511d719727ef9a17cec39971353b91519', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxf288f27511d719727ef9a17cec39971353b91519', 'cx53ebde99a0b8d4dc9d30fbe6a6cff77c75b693f6', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxf288f27511d719727ef9a17cec39971353b91519', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx53ebde99a0b8d4dc9d30fbe6a6cff77c75b693f6', 'cxecbf641b35396cc360d02183b9c10f5d5f65536f', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxf288f27511d719727ef9a17cec39971353b91519', 'indexed': ['